# Politician Activity on Wikipedia

The parameters in the cell below can be adjusted to explore otherPoliticians and time frames.

### How to explore other politicians?
The ***politician_ID*** is an internal identifier that connects the different social media accounts. You can [use this other notebook](../Politicians.ipynb?autorun=true) to get other the identifiers of other politicians.

***Alternatively***, you can direcly use the [Politicians API](http://mediamonitoring.gesis.org/api/Politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
politician_id = 1928
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using APIs
### B.1 Using the SMM Politician API

In [ ]:
from smm_wrapper import SMMPoliticians

# create an instance to the smm wrapper
smm = SMMPoliticians()

# request the politician from the API
politician = smm.dv.get_one(politician_id)

# using the api to get the change objects
wiki_chobs = smm.dv.wikipedia(_id=politician_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation)
wiki_chobs = wiki_chobs.groupby('date').agg({'chobs': 'sum'}).reset_index()

### B.2 Using the Wikiwho API

In [ ]:
from wikiwho_wrapper import WikiWho
import pandas as pd

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wiki_id, start=from_date, end=to_date) 
                      for wiki_id in politician['wp_ids'])
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data['date'] = pd.to_datetime(wiki_data['year_month'])
wiki_data = wiki_data.groupby('date')['conflict','elegibles','undos'].sum().reset_index()
wiki_data['conflict_score'] = wiki_data['conflict'] / wiki_data['elegibles']
wiki_data.fillna(0, inplace=True)

### B.3 Using the Wikimedia API

In [ ]:
import requests

# using wikimedia to extract views
session = requests.Session()
vurl = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{wp_title}/daily/" + f"{from_date.replace('-','')}/{to_date.replace('-','')}"
views = pd.concat([pd.DataFrame(session.get(url=vurl.format(wp_title=wp_title)).json()['items']) 
           for wp_title in politician['wp_titles']])
views['timestamp']=pd.to_datetime(views['timestamp'], format='%Y%m%d%H')

# weekly or monthly aggregation of the data
if aggregation  == 'week':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='W-SUN')])['views'].sum().reset_index().sort_values('timestamp')
    views['timestamp'] = views['timestamp'] - pd.Timedelta(days=6)
    
elif aggregation  == 'month':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='MS')])['views'].sum().reset_index().sort_values('timestamp')

## C. Plotting
### C.1 Plot Wikipedia Activity

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=views['timestamp'], y=views['views'], name='Views', line_shape='spline'),
             go.Scatter(x=wiki_chobs['date'], y=wiki_chobs['chobs'], name='Changes', yaxis='y2', line_shape='spline')], 
    "layout": go.Layout( title='Wikipedia Activity', yaxis=dict(title='Views'),
    yaxis2=dict(title='Changes', overlaying='y', side='right'))
})

### C.2 Plot Wikipedia Disagreement

In [ ]:
plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_data['date'], y=wiki_data['undos'], name='Undos', line_shape='spline'),
            go.Scatter(x=wiki_data['date'], y=wiki_data['conflict_score'], name='Conflict', line_shape='spline', yaxis='y2')], 
    "layout": go.Layout(title='Wikipedia Disagreement', yaxis=dict(title='Undos'),
    yaxis2=dict(title='Conflict', overlaying='y',side='right'))
})